In [235]:
import pandas as pd
import numpy as np
import csv

# Import data

In [236]:
DATA_FOLDER = "competition-data/"
DATA_TEST = "test.csv"
DATA_TRAIN = "train.csv"

In [237]:
train_path = DATA_FOLDER + DATA_TRAIN
train_data = pd.read_csv(train_path, na_values=[-999.000, 0.0])

In [238]:
test_path = DATA_FOLDER + DATA_TEST
test_data = pd.read_csv(test_path, na_values=[-999.000, 0.0])

In [239]:
train_data.head()

,Id,Prediction,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,...,PRI_met_phi,PRI_met_sumet,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt
0,100000,s,138.470,51.655,97.827,27.980,0.91,124.711,2.666,3.064,...,-0.277,258.733,2.0,67.435,2.150,0.444,46.062,1.24,-2.475,113.497
1,100001,b,160.937,68.768,103.235,48.146,NaN,NaN,NaN,3.473,...,-1.916,164.546,1.0,46.226,0.725,1.158,NaN,NaN,NaN,46.226
2,100002,b,NaN,162.172,125.953,35.635,NaN,NaN,NaN,3.148,...,-2.186,260.414,1.0,44.251,2.053,-2.028,NaN,NaN,NaN,44.251
3,100003,b,143.905,81.417,80.943,0.414,NaN,NaN,NaN,3.310,...,0.060,86.062,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100004,b,175.864,16.915,134.805,16.405,NaN,NaN,NaN,3.891,...,-0.871,53.131,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [240]:
# constants
CATEGORICAL_FIELDS = ['PRI_jet_num']

In [241]:
y = rename_y(1, -1, train_data['Prediction'])


In [242]:
# only the features
train_data = train_data.loc[:, (train_data.columns != "Id") & (train_data.columns != "Prediction")]
test_data = test_data.loc[:, (test_data.columns != "Id") & (test_data.columns != "Prediction")]

In [243]:
train_data.isnull().sum()/250000

DER_mass_MMC                   0.152456
DER_mass_transverse_met_lep    0.000012
DER_mass_vis                   0.000000
DER_pt_h                       0.000164
DER_deltaeta_jet_jet           0.709852
DER_mass_jet_jet               0.709828
DER_prodeta_jet_jet            0.710060
DER_deltar_tau_lep             0.000000
DER_pt_tot                     0.000156
DER_sum_pt                     0.000000
DER_pt_ratio_lep_tau           0.000000
DER_met_phi_centrality         0.000212
DER_lep_eta_centrality         0.772836
PRI_tau_pt                     0.000000
PRI_tau_eta                    0.000000
PRI_tau_phi                    0.000128
PRI_lep_pt                     0.000000
PRI_lep_eta                    0.000140
PRI_lep_phi                    0.000132
PRI_met                        0.000000
PRI_met_phi                    0.000176
PRI_met_sumet                  0.000000
PRI_jet_num                    0.399652
PRI_jet_leading_pt             0.399652
PRI_jet_leading_eta            0.399756


# clean data

# median and category (most frequent policy)

In [251]:
def median_rep_data(train_data, test_data):
    complete_data = pd.concat([test_data, train_data])
    complete_data_median = complete_data.median()
    median_rep_data_train = train_data.fillna(complete_data_median)
    median_rep_data_test = test_data.fillna(complete_data_median)
    
    return (median_rep_data_train, median_rep_data_test)

def categorical_rep_data(column):
    column_wo_nan = column.dropna()
    v = column_wo_nan.value_counts().idxmax()
    
    return column.fillna(v)

In [252]:
(median_rep_data_train, median_rep_data_test) = median_rep_data(train_data, test_data)

for c in CATEGORICAL_FIELDS:
    median_rep_data_train[c] = categorical_rep_data(train_data[c])
    median_rep_data_test[c] = categorical_rep_data(test_data[c])

In [253]:
median_rep_data_train.head()

,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,DER_pt_tot,DER_sum_pt,...,PRI_met_phi,PRI_met_sumet,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt
0,138.470,51.655,97.827,27.980,0.910,124.711,2.666,3.064,41.928,197.760,...,-0.277,258.733,2.0,67.435,2.150,0.444,46.062,1.240,-2.475,113.497
1,160.937,68.768,103.235,48.146,2.102,226.193,-0.246,3.473,2.078,125.157,...,-1.916,164.546,1.0,46.226,0.725,1.158,47.993,-0.012,-0.021,46.226
2,112.501,162.172,125.953,35.635,2.102,226.193,-0.246,3.148,9.336,197.814,...,-2.186,260.414,1.0,44.251,2.053,-2.028,47.993,-0.012,-0.021,44.251
3,143.905,81.417,80.943,0.414,2.102,226.193,-0.246,3.310,0.414,75.968,...,0.060,86.062,1.0,65.760,0.001,-0.044,47.993,-0.012,-0.021,92.040
4,175.864,16.915,134.805,16.405,2.102,226.193,-0.246,3.891,16.405,57.983,...,-0.871,53.131,1.0,65.760,0.001,-0.044,47.993,-0.012,-0.021,92.040


In [75]:
def rename_y(higgs_value, background_value, named_y):
    return pd.Series(list(map(lambda x : higgs_value if x=='s' else background_value, named_y)))

In [175]:
train_data['Prediction'].value_counts()

b    164333
s     85667
Name: Prediction, dtype: int64

# balance output/input

In [255]:
features = median_rep_data_train[median_rep_data_train.columns[2:-1]].apply(lambda x: x.corr(y)).sort_values().where(lambda x : abs(x) > 0).dropna().index

In [256]:
sanitized_train_data = median_rep_data_train[features]
sanitized_test_data = median_rep_data_test[features]

In [257]:
def extract_tx(sanitized_train_data, sanitized_test_data):
    tx_train = sanitized_train_data[sanitized_train_data.columns[2:]].values
    tx_test = sanitized_test_data[sanitized_test_data.columns[2:]].values
    
    return (tx_train, tx_test)

In [258]:
(tx_train, tx_test) = extract_tx(sanitized_train_data, sanitized_test_data)

In [259]:
def init_w(tx):
    return np.random.rand(tx.shape[1])

In [260]:
initial_w = init_w(tx_train)
initial_w

array([ 0.59265594,  0.03120041,  0.04328736,  0.76829106,  0.04707385,
        0.5826421 ,  0.01867113,  0.64860029,  0.07359482,  0.29635716,
        0.23869248,  0.93118973,  0.38763875,  0.3637642 ,  0.59855078,
        0.34380471,  0.20047415,  0.76984161,  0.97921889,  0.42688736,
        0.38239268,  0.32253567,  0.30520061,  0.39644763,  0.96423429])

# Algorithms

In [261]:
def compute_gradient(y, tx, w):
    # Compute the gradient and loss using MSE

    N = len(y)
    e = y - tx.dot(w)
    gradient = -1/N * tx.T.dot(e)

    return gradient

def compute_loss(y, tx, w):
    # Compute the gradient and loss using MSE

    N = len(y)
    e = y - tx.dot(w)
    loss = 1/(2*N) * np.sum(e**2, axis=0)

    return loss

def least_squares_GD(y, tx, initial_w, max_iters, gamma):
    # Linear regression using gradient descent

    w = initial_w
    for n_iter in range(max_iters):
        # compute gradient and loss
        gradient = compute_gradient(y, tx, w)
        loss = compute_loss(y, tx, w)
        # update w by gradient
        w = w - gamma/(n_iter+1) * gradient

    return (w, loss)

def standardize(x):
    
    centered_data = x - np.mean(x, axis=0)
    std_data = centered_data / np.std(centered_data, axis=0)
    
    return std_data

def least_squares(y, tx):
    # Least squares regression using normal equations


    w = np.linalg.solve(tx.T.dot(tx), tx.T.dot(y))
    loss = compute_loss(y, tx, w)

    return (w, loss)

def build_poly_feature(x, degree):
    """polynomial basis functions for input data x, for j=0 up to j=degree."""
    # ***************************************************
    # INSERT YOUR CODE HERE
    # polynomial basis function: TODO
    # this function should return the matrix formed
    # by applying the polynomial basis to the input data
    # ***************************************************
    
    if x.ndim == 1:
        x = x[:,np.newaxis]
    
    polynomial_basis = x**0
    
    for j in range(1,degree+1):
        #polynomial_basis = np.concatenate((polynomial_basis,x**j), axis=1)
        polynomial_basis = np.hstack((polynomial_basis, x**j))
    return polynomial_basis

def build_poly_tx(tx, degree):
    tx_polynomial = build_poly_feature(tx.T[0].T, degree)
    for c in tx.T[1:]:
        tx_polynomial = np.hstack((tx_polynomial, build_poly_feature(c.T, degree)))
    return tx_polynomial

In [262]:
tx_train = standardize(tx_train)
tx_test = standardize(tx_test)

## least_squares_GD

In [163]:
%time (w_gd, loss_gd) = least_squares_GD(y, tx_train, initial_w, 10000, 0.5)

CPU times: user 7.27 s, sys: 813 ms, total: 8.09 s
Wall time: 1.64 s


In [164]:
print(w_gd, "\n\nloss: ", loss_gd)

[ 0.10574866  0.09236508  0.03878155 -0.0771191   0.02303186  0.10582769
  0.04464807  0.06094469  0.12317494  0.04415956  0.01757313  0.10521949
  0.00699009 -0.06091452  0.12234318  0.20546308 -0.33254534  0.08169881
 -0.05244991  0.19386155  0.2695795 ] 

loss:  0.48501168665196565


## least_squares

In [263]:
%time (w_ls, loss_ls) = least_squares(y, tx_train)

CPU times: user 177 ms, sys: 66.7 ms, total: 243 ms
Wall time: 72.3 ms


In [264]:
print(w_ls, "\n\nloss: ", loss_ls)

[  1.60045409e-02  -4.57194014e-02  -2.65026392e-01  -1.44003090e-03
  -1.36729786e-03  -1.11545323e-03   1.05392421e-04   1.80478717e-03
  -2.32591470e-04  -8.82607480e-04   2.50842521e-03   3.08703941e-03
   3.28001504e-01  -6.61817986e-02  -2.18931999e-02   1.91787915e-02
  -4.21051267e-02   7.29656277e-02  -5.37747062e-02   3.05388700e-02
   2.26800837e-01   7.27046612e-02   7.34225793e-02   2.78089204e-01
   1.60531785e-01] 

loss:  0.4088868438346996


## polynomial_regression

In [186]:
tx_train_poly.shape
initial_w.shape

(42,)

In [185]:
degree = 1
tx_train_poly = build_poly_tx(tx_train, degree)
tx_test_poly = build_poly_tx(tx_test, degree)
initial_w = init_w(tx_train_poly)
initial_w

array([ 0.5150735 ,  0.78564416,  0.69320694,  0.95723246,  0.27271904,
        0.22039559,  0.06263581,  0.770136  ,  0.1481901 ,  0.46454345,
        0.04145503,  0.14054985,  0.0440912 ,  0.64101959,  0.43980429,
        0.35140236,  0.25071974,  0.06248324,  0.00717547,  0.14856598,
        0.6184201 ,  0.25228335,  0.69793237,  0.83583985,  0.10272617,
        0.48626491,  0.97490694,  0.26793409,  0.8909672 ,  0.47688436,
        0.10409208,  0.80195907,  0.53746707,  0.19428553,  0.69638683,
        0.10441173,  0.09380573,  0.80300296,  0.56490464,  0.61873557,
        0.85332171,  0.07427372])

In [189]:
%time (w_poly, loss_poly) = least_squares_GD(y, tx_train_poly, initial_w, 10000, 0.5)

CPU times: user 15min 4s, sys: 1min 5s, total: 16min 10s
Wall time: 3min 24s


In [188]:
print(w_poly, "\n\nloss: ", loss_poly)

[ 0.0900894  -0.03960436  0.26822284  0.03914141 -0.15226505  0.0162004
 -0.36234828  0.0322052  -0.27679399  0.01882001 -0.38352906  0.04037505
 -0.38089289  0.02002482  0.0148202  -0.04526306 -0.17426436  0.04511735
 -0.41780862  0.02120847  0.193436   -0.12685992  0.27294827  0.24750619
 -0.32225792  0.01953835  0.54992285  0.02534631  0.46598311  0.13019426
 -0.32089202  0.24144211  0.11248297 -0.18436212  0.27140273  0.04185665
 -0.33117836  0.0652517   0.13992055  0.12188187  0.42833762  0.12855381] 

loss:  0.41908336652603734


In [127]:
def predict_labels(weights, data):
    """Generates class predictions given weights, and a test data matrix"""
    y_pred = np.dot(data, weights)
    y_pred[np.where(y_pred <= 0)] = -1
    y_pred[np.where(y_pred > 0)] = 1
    
    return y_pred

In [265]:
y_pred = predict_labels(w_ls, tx_test)
y_pred

array([-1., -1., -1., ...,  1.,  1., -1.])

# Submission

In [106]:
def create_csv_submission(ids, y_pred, name):
    """
    Creates an output file in csv format for submission to kaggle
    Arguments: ids (event ids associated with each prediction)
               y_pred (predicted class labels)
               name (string name of .csv output file to be created)
    """
    with open(name, 'w') as csvfile:
        fieldnames = ['Id', 'Prediction']
        writer = csv.DictWriter(csvfile, delimiter=",", fieldnames=fieldnames)
        writer.writeheader()
        for r1, r2 in zip(ids, y_pred):
            writer.writerow({'Id':int(r1),'Prediction':int(r2)})

In [266]:
create_csv_submission(sanitized_test_data_complete['Id'], y_pred, "test5.csv")

Test n : algorithm / features / y / w
- - - - - - - - - - - - - - - - - - - 
Test 1 : least_squares / all features standardized / y = -1,1 / random init_w

Test 2 : least_squares / corr > 0.1 features standardized / y = -1,1 / random init_w

Test 3 : least_squares_GD(10000,0.5) / all features standardized / y = -1,1 / random init_w

Test 4 : least_squares_GD(10000,0.5) / all features standardized / y = -1,1 / random init_w / poly, degree=1

Test 5 : least_squares / all features standardized / y = -1,1 / random init_w / median + categorical

## Further work

- balance output (batch numpy)
- median and category
- features engineering : features d'intéraction
- logistic regression 